In [1]:
import pandas as pd
import numpy as np


from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

from catboost import CatBoostClassifier

from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer


import os
print(os.listdir("input"))

['PAGE.csv', 'device_data.csv', 'pageviews.csv', 'CONTENT_CATEGORY.csv', 'pageviews_complemento.csv', 'conversiones.csv', 'CONTENT_CATEGORY_TOP.csv', 'sampleSubmission.csv', 'CONTENT_CATEGORY_BOTTOM.csv', 'SITE_ID.csv']


## Creación de datasets 

Los datasets serán creados de la misma forma que en el notebook del benchmark

In [2]:
data = pd.concat([
       pd.read_csv("input/pageviews.csv", parse_dates=["FEC_EVENT"]),
       pd.read_csv("input/pageviews_complemento.csv", parse_dates=["FEC_EVENT"])
])

# Test
X_test = []
for c in data.drop(["USER_ID", "FEC_EVENT"], axis=1).columns:
    print("haciendo", c)
    temp = pd.crosstab(data.USER_ID, data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_test.append(temp.apply(lambda x: x / x.sum(), axis=1))
X_test = pd.concat(X_test, axis=1)

# Train
data = data[data.FEC_EVENT.dt.month < 10]
X_train = []
for c in data.drop(["USER_ID", "FEC_EVENT"], axis=1).columns:
    print("haciendo", c)
    temp = pd.crosstab(data.USER_ID, data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_train.append(temp.apply(lambda x: x / x.sum(), axis=1))
X_train = pd.concat(X_train, axis=1)

features = list(set(X_train.columns).intersection(set(X_test.columns)))
X_train = X_train[features]
X_test = X_test[features]

y_prev = pd.read_csv("input/conversiones.csv")
y_train = pd.Series(0, index=X_train.index)
idx = set(y_prev[y_prev.mes >= 10].USER_ID.unique()).intersection(
        set(X_train.index))
y_train.loc[list(idx)] = 1

/home/rodrigo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


haciendo CONTENT_CATEGORY
haciendo CONTENT_CATEGORY_BOTTOM
haciendo CONTENT_CATEGORY_TOP
haciendo ON_SITE_SEARCH_TERM
haciendo PAGE
haciendo SITE_ID
haciendo CONTENT_CATEGORY
haciendo CONTENT_CATEGORY_BOTTOM
haciendo CONTENT_CATEGORY_TOP
haciendo ON_SITE_SEARCH_TERM
haciendo PAGE
haciendo SITE_ID


## Definición de funciones útiles

In [3]:
def model_auc_score(model, X_t, y_t, X_v, y_v):
    model.fit(X_t, y_t)
    y_pred_t = model.predict_proba(X_t)[:,1]
    y_pred_v = model.predict_proba(X_v)[:,1]
    auc_t = roc_auc_score(y_t, y_pred_t)
    auc_v = roc_auc_score(y_v, y_pred_v)
    return auc_t, auc_v

def report_model(label, auc_t, auc_v):
    print('------- ' + label + '  --------')
    print('Training auc:   ' + str(auc_t))
    print('Validation auc: ' + str(auc_v))
    


## CATBOOST

In [9]:


SEED = 18 
x_train = X_train.values
y_tr = y_train.values

# dividiendo el dataset
X_t, X_v, y_t, y_v = train_test_split(x_train, y_tr, test_size=0.20, random_state=SEED)


params = {'depth':[3,1,2,6,4,5,7,8,9,10],
          'iterations':[250,100,500,1000],
          'learning_rate':[0.03,0.001,0.01,0.1,0.2,0.3], 
          'l2_leaf_reg':[3,1,5,10,100],
          'border_count':[32,5,10,20,50,100,200]}


cat = CatBoostClassifier(random_seed = SEED)
random = RandomizedSearchCV(cat,params, cv=10, scoring = 'roc_auc')

# catboost



#auc_t, auc_v = model_auc_score(cat, X_t, y_t, X_v, y_v)
#report_model('Catboost', auc_t, auc_v)

In [10]:
random.fit(X_t,y_t)

0:	learn: 0.3429976	total: 1.36s	remaining: 22m 34s
1:	learn: 0.2215269	total: 2.6s	remaining: 21m 37s
2:	learn: 0.1555876	total: 3.85s	remaining: 21m 18s
3:	learn: 0.1334564	total: 5.07s	remaining: 21m 1s
4:	learn: 0.1257129	total: 6.27s	remaining: 20m 48s
5:	learn: 0.1188371	total: 7.61s	remaining: 21m
6:	learn: 0.1173022	total: 8.97s	remaining: 21m 12s
7:	learn: 0.1152196	total: 10.3s	remaining: 21m 21s
8:	learn: 0.1122370	total: 11.6s	remaining: 21m 18s
9:	learn: 0.1091619	total: 13s	remaining: 21m 25s
10:	learn: 0.1057153	total: 14.4s	remaining: 21m 35s
11:	learn: 0.1052166	total: 15.8s	remaining: 21m 39s
12:	learn: 0.1031313	total: 17.2s	remaining: 21m 47s
13:	learn: 0.1018562	total: 18.5s	remaining: 21m 44s
14:	learn: 0.1013008	total: 19.7s	remaining: 21m 34s
15:	learn: 0.1009773	total: 20.9s	remaining: 21m 26s
16:	learn: 0.1007022	total: 22.2s	remaining: 21m 23s
17:	learn: 0.0996652	total: 23.4s	remaining: 21m 17s
18:	learn: 0.0989662	total: 24.6s	remaining: 21m 10s
19:	learn: 

154:	learn: 0.0281742	total: 3m 15s	remaining: 17m 45s
155:	learn: 0.0278632	total: 3m 16s	remaining: 17m 43s
156:	learn: 0.0276671	total: 3m 17s	remaining: 17m 41s
157:	learn: 0.0273535	total: 3m 18s	remaining: 17m 40s
158:	learn: 0.0266737	total: 3m 20s	remaining: 17m 39s
159:	learn: 0.0263569	total: 3m 21s	remaining: 17m 39s
160:	learn: 0.0255248	total: 3m 23s	remaining: 17m 38s
161:	learn: 0.0254328	total: 3m 24s	remaining: 17m 37s
162:	learn: 0.0253139	total: 3m 26s	remaining: 17m 37s
163:	learn: 0.0248656	total: 3m 27s	remaining: 17m 36s
164:	learn: 0.0246222	total: 3m 28s	remaining: 17m 35s
165:	learn: 0.0242358	total: 3m 29s	remaining: 17m 33s
166:	learn: 0.0238732	total: 3m 30s	remaining: 17m 31s
167:	learn: 0.0237141	total: 3m 32s	remaining: 17m 30s
168:	learn: 0.0236871	total: 3m 33s	remaining: 17m 28s
169:	learn: 0.0235194	total: 3m 34s	remaining: 17m 27s
170:	learn: 0.0233051	total: 3m 35s	remaining: 17m 25s
171:	learn: 0.0230063	total: 3m 36s	remaining: 17m 23s
172:	learn

305:	learn: 0.0090531	total: 6m 24s	remaining: 14m 32s
306:	learn: 0.0090440	total: 6m 26s	remaining: 14m 31s
307:	learn: 0.0090233	total: 6m 27s	remaining: 14m 30s
308:	learn: 0.0089521	total: 6m 28s	remaining: 14m 29s
309:	learn: 0.0088667	total: 6m 29s	remaining: 14m 27s
310:	learn: 0.0088605	total: 6m 31s	remaining: 14m 27s
311:	learn: 0.0088526	total: 6m 32s	remaining: 14m 26s
312:	learn: 0.0088208	total: 6m 34s	remaining: 14m 25s
313:	learn: 0.0088088	total: 6m 35s	remaining: 14m 24s
314:	learn: 0.0087953	total: 6m 37s	remaining: 14m 23s
315:	learn: 0.0087707	total: 6m 38s	remaining: 14m 22s
316:	learn: 0.0087552	total: 6m 39s	remaining: 14m 21s
317:	learn: 0.0087522	total: 6m 40s	remaining: 14m 19s
318:	learn: 0.0085057	total: 6m 42s	remaining: 14m 18s
319:	learn: 0.0084349	total: 6m 43s	remaining: 14m 16s
320:	learn: 0.0084301	total: 6m 44s	remaining: 14m 15s
321:	learn: 0.0083706	total: 6m 45s	remaining: 14m 14s
322:	learn: 0.0082484	total: 6m 46s	remaining: 14m 12s
323:	learn

455:	learn: 0.0047932	total: 9m 39s	remaining: 11m 30s
456:	learn: 0.0047849	total: 9m 40s	remaining: 11m 29s
457:	learn: 0.0047820	total: 9m 41s	remaining: 11m 28s
458:	learn: 0.0047765	total: 9m 42s	remaining: 11m 26s
459:	learn: 0.0047714	total: 9m 43s	remaining: 11m 25s
460:	learn: 0.0047704	total: 9m 45s	remaining: 11m 24s
461:	learn: 0.0047489	total: 9m 46s	remaining: 11m 22s
462:	learn: 0.0047395	total: 9m 47s	remaining: 11m 21s
463:	learn: 0.0047016	total: 9m 49s	remaining: 11m 20s
464:	learn: 0.0046816	total: 9m 50s	remaining: 11m 19s
465:	learn: 0.0046561	total: 9m 51s	remaining: 11m 18s
466:	learn: 0.0046442	total: 9m 52s	remaining: 11m 16s
467:	learn: 0.0046263	total: 9m 54s	remaining: 11m 15s
468:	learn: 0.0045916	total: 9m 55s	remaining: 11m 14s
469:	learn: 0.0045854	total: 9m 56s	remaining: 11m 12s
470:	learn: 0.0045758	total: 9m 57s	remaining: 11m 11s
471:	learn: 0.0045366	total: 9m 59s	remaining: 11m 10s
472:	learn: 0.0045253	total: 10m	remaining: 11m 8s
473:	learn: 0.

605:	learn: 0.0033417	total: 12m 47s	remaining: 8m 19s
606:	learn: 0.0033413	total: 12m 49s	remaining: 8m 17s
607:	learn: 0.0033365	total: 12m 50s	remaining: 8m 16s
608:	learn: 0.0033356	total: 12m 51s	remaining: 8m 15s
609:	learn: 0.0033353	total: 12m 52s	remaining: 8m 13s
610:	learn: 0.0033352	total: 12m 53s	remaining: 8m 12s
611:	learn: 0.0033340	total: 12m 54s	remaining: 8m 11s
612:	learn: 0.0033315	total: 12m 56s	remaining: 8m 9s
613:	learn: 0.0033293	total: 12m 57s	remaining: 8m 8s
614:	learn: 0.0033264	total: 12m 58s	remaining: 8m 7s
615:	learn: 0.0033260	total: 12m 59s	remaining: 8m 6s
616:	learn: 0.0033254	total: 13m 1s	remaining: 8m 5s
617:	learn: 0.0033055	total: 13m 2s	remaining: 8m 3s
618:	learn: 0.0032969	total: 13m 3s	remaining: 8m 2s
619:	learn: 0.0032966	total: 13m 4s	remaining: 8m 1s
620:	learn: 0.0032930	total: 13m 6s	remaining: 7m 59s
621:	learn: 0.0032620	total: 13m 7s	remaining: 7m 58s
622:	learn: 0.0032619	total: 13m 8s	remaining: 7m 57s
623:	learn: 0.0032598	tot

...

19:	learn: 0.1163829	total: 686ms	remaining: 2.74s
20:	learn: 0.1162207	total: 730ms	remaining: 2.75s
21:	learn: 0.1160735	total: 760ms	remaining: 2.7s
22:	learn: 0.1159792	total: 791ms	remaining: 2.65s
23:	learn: 0.1157500	total: 823ms	remaining: 2.6s
24:	learn: 0.1155552	total: 853ms	remaining: 2.56s
25:	learn: 0.1151533	total: 885ms	remaining: 2.52s
26:	learn: 0.1142482	total: 928ms	remaining: 2.51s
27:	learn: 0.1140750	total: 959ms	remaining: 2.47s
28:	learn: 0.1132634	total: 988ms	remaining: 2.42s
29:	learn: 0.1125917	total: 1.02s	remaining: 2.38s
30:	learn: 0.1123760	total: 1.05s	remaining: 2.34s
31:	learn: 0.1121930	total: 1.08s	remaining: 2.3s
32:	learn: 0.1117744	total: 1.11s	remaining: 2.25s
33:	learn: 0.1116609	total: 1.14s	remaining: 2.21s
34:	learn: 0.1114938	total: 1.18s	remaining: 2.18s
35:	learn: 0.1114102	total: 1.22s	remaining: 2.17s
36:	learn: 0.1112468	total: 1.26s	remaining: 2.14s
37:	learn: 0.1109863	total: 1.33s	remaining: 2.18s
38:	learn: 0.1108559	total: 1.37s	

82:	learn: 0.1039578	total: 2.9s	remaining: 595ms
83:	learn: 0.1036378	total: 2.94s	remaining: 560ms
84:	learn: 0.1032989	total: 2.97s	remaining: 525ms
85:	learn: 0.1032788	total: 3s	remaining: 489ms
86:	learn: 0.1031375	total: 3.03s	remaining: 453ms
87:	learn: 0.1029702	total: 3.06s	remaining: 417ms
88:	learn: 0.1028447	total: 3.09s	remaining: 382ms
89:	learn: 0.1027136	total: 3.12s	remaining: 347ms
90:	learn: 0.1026107	total: 3.16s	remaining: 312ms
91:	learn: 0.1023208	total: 3.2s	remaining: 278ms
92:	learn: 0.1022595	total: 3.24s	remaining: 244ms
93:	learn: 0.1021662	total: 3.31s	remaining: 211ms
94:	learn: 0.1020691	total: 3.37s	remaining: 178ms
95:	learn: 0.1018542	total: 3.41s	remaining: 142ms
96:	learn: 0.1014913	total: 3.44s	remaining: 106ms
97:	learn: 0.1013308	total: 3.47s	remaining: 70.8ms
98:	learn: 0.1010858	total: 3.5s	remaining: 35.3ms
99:	learn: 0.1009604	total: 3.53s	remaining: 0us
0:	learn: 0.4349425	total: 36ms	remaining: 3.57s
1:	learn: 0.3010650	total: 74ms	remaini

48:	learn: 0.1102393	total: 1.56s	remaining: 1.63s
49:	learn: 0.1100645	total: 1.59s	remaining: 1.59s
50:	learn: 0.1099529	total: 1.62s	remaining: 1.55s
51:	learn: 0.1097288	total: 1.66s	remaining: 1.53s
52:	learn: 0.1096389	total: 1.68s	remaining: 1.49s
53:	learn: 0.1095530	total: 1.72s	remaining: 1.46s
54:	learn: 0.1093205	total: 1.75s	remaining: 1.43s
55:	learn: 0.1091633	total: 1.78s	remaining: 1.4s
56:	learn: 0.1090117	total: 1.82s	remaining: 1.37s
57:	learn: 0.1088906	total: 1.85s	remaining: 1.34s
58:	learn: 0.1086716	total: 1.88s	remaining: 1.31s
59:	learn: 0.1085055	total: 1.91s	remaining: 1.27s
60:	learn: 0.1083345	total: 1.94s	remaining: 1.24s
61:	learn: 0.1080923	total: 1.97s	remaining: 1.21s
62:	learn: 0.1079763	total: 2s	remaining: 1.18s
63:	learn: 0.1078083	total: 2.04s	remaining: 1.14s
64:	learn: 0.1076935	total: 2.06s	remaining: 1.11s
65:	learn: 0.1073213	total: 2.1s	remaining: 1.08s
66:	learn: 0.1069286	total: 2.13s	remaining: 1.05s
67:	learn: 0.1066062	total: 2.16s	re

111:	learn: 0.1126944	total: 14s	remaining: 17.2s
112:	learn: 0.1125126	total: 14.1s	remaining: 17.1s
113:	learn: 0.1124976	total: 14.2s	remaining: 17s
114:	learn: 0.1124608	total: 14.3s	remaining: 16.8s
115:	learn: 0.1123864	total: 14.4s	remaining: 16.7s
116:	learn: 0.1123472	total: 14.6s	remaining: 16.6s
117:	learn: 0.1122735	total: 14.7s	remaining: 16.4s
118:	learn: 0.1122514	total: 14.8s	remaining: 16.3s
119:	learn: 0.1121816	total: 14.9s	remaining: 16.2s
120:	learn: 0.1121222	total: 15s	remaining: 16s
121:	learn: 0.1120292	total: 15.2s	remaining: 15.9s
122:	learn: 0.1119126	total: 15.3s	remaining: 15.8s
123:	learn: 0.1118384	total: 15.4s	remaining: 15.7s
124:	learn: 0.1117892	total: 15.5s	remaining: 15.5s
125:	learn: 0.1116072	total: 15.7s	remaining: 15.4s
126:	learn: 0.1115796	total: 15.8s	remaining: 15.3s
127:	learn: 0.1114124	total: 15.9s	remaining: 15.2s
128:	learn: 0.1113552	total: 16s	remaining: 15s
129:	learn: 0.1113252	total: 16.1s	remaining: 14.9s
130:	learn: 0.1112396	to

RandomizedSearchCV(cv=10, error_score='raise-deprecating',
                   estimator=<catboost.core.CatBoostClassifier object at 0x7fdf372e3668>,
                   iid='warn', n_iter=10, n_jobs=None,
                   param_distributions={'border_count': [32, 5, 10, 20, 50, 100,
                                                         200],
                                        'depth': [3, 1, 2, 6, 4, 5, 7, 8, 9,
                                                  10],
                                        'iterations': [250, 100, 500, 1000],
                                        'l2_leaf_reg': [3, 1, 5, 10, 100],
                                        'learning_rate': [0.03, 0.001, 0.01,
                                                          0.1, 0.2, 0.3]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring='roc_auc', verbose=0)

In [11]:
y_pred_t = random.predict_proba(X_t)[:,1]
y_pred_v = random.predict_proba(X_v)[:,1]
auc_t = roc_auc_score(y_t, y_pred_t)
auc_v = roc_auc_score(y_v, y_pred_v)
report_model('Catboost', auc_t, auc_v)

------- Catboost  --------
Training auc:   0.9092990699332638
Validation auc: 0.8563117699910153


In [12]:
cat_proba = random.predict_proba(X_test)
len(cat_proba)

11676

In [13]:
sm = pd.read_csv("input/sampleSubmission.csv")
sm['SCORE'] = cat_proba
sm.sort_values(by = 'SCORE', ascending=True)
sm.to_csv("submission_random.csv", index=False)

